In [38]:
import numpy as np
import pandas as pd
import re 
import matplotlib.pyplot as plt 
import sys

In [39]:
#Input file must be given to run the code.
#For Example Run(in terminal) 'python3 Bioinformatics input.xlsx'

#Filename =sys.argv[1] # Read input file 

df = pd.read_excel('input.xlsx')# Read EXCEL Files and storing in database 'df'
data= pd.DataFrame()# Creating New Data Frame 

In [40]:
# Storing Usefull information From the database 'df' 
data['Entry']=df['Entry'] #Storing Entry
data['Entry name']=df['Entry name']  #Storing Entry Name
data['Protein names']=df['Protein names']  #Storing Protein Names
data['Gene names']=df['Gene names'] # Storing Gene Names
data['Length']=df['Length'] # Storing Length

In [41]:
data['Organism']=df['Organism'] # Storing Type of Organism
data['Mass']=df['Mass'] #Storing Mass
data['Status']=df['Status']

In [42]:
#Now removing the extra data we dont need and containg only organism 'Human'
columns=data[data['Organism'].str.contains("Human")] #Selecting only organism which contains HUMANS
data= columns

In [43]:
data['Disulfide bond']=df['Disulfide bond'] # Storing Disulfide bond 
data['Glycosylation']= df['Glycosylation']# Storing Glycosylation Position

In [44]:
data=data.dropna().reset_index(drop=True) # Drop all rows which contains Not a number and reset the index 

In [45]:
# Working on getting the relative positions of the Disulfide bond
disulfide_column= data['Disulfide bond'] # Storing data temporary as 'disulfide_column'
def getdata(newdata):
    return re.findall('\d+ \d+',newdata)

data['Disulfide bond']= disulfide_column.apply(getdata) # Function call which gives all the positons 

In [46]:
#Working on getting the positions of N-linked Glycosylation
Glyco_data= data['Glycosylation'] # Storing data temporary as 'Glyco_data'

def getGlycoNLinked(data):   #Making a Function to get the positions
    return re.findall('\d+\s* N-linked',data) # The return data will contain 'position and N-linked pattern'
data['Glycosylation']= Glyco_data.apply(getGlycoNLinked) # the return data will be like '49 N-linked'

In [47]:
#Now Removing the extra word 'N-linked' and getting all positions 
temp_data= data['Glycosylation'].astype(str)
def getValue(data):
    return re.findall('\d+',data) # return all positions
data['Glycosylation']= temp_data.apply(getValue) 

In [48]:
a= pd.DataFrame()
b= pd.DataFrame()

a =pd.Series(data['Glycosylation'])
b =pd.Series(data['Disulfide bond'])

lenB=b.str.len()
lenA=a.str.len()
rel_pos = pd.Series(a.size,dtype=np.str)

for i in range(a.size):
    rel_pos[i]=''

In [49]:
i=0;
while i<b.size:
    for j in range(lenA[i]):
        Position=int(a[i][j])
        rel_pos[i]=rel_pos[i]+str(a[i][j])+'{|'
        for k in range(lenB[i]):
            split = re.findall('\d+',b[i][k]) 
            firstPair= int(split[0])
            secondPair=int(split[1])
            if Position< firstPair:
                rel_pos[i] = rel_pos[i]+'o'+str(Position-firstPair)
                rel_pos[i] = rel_pos[i]+'o'+str(Position-secondPair)
            elif Position>= firstPair and Position<=secondPair:
                rel_pos[i]=  rel_pos[i]+'i' + str(Position-firstPair)
                rel_pos[i] = rel_pos[i]+'i'+ str(Position-secondPair)
            else:
                rel_pos[i] = rel_pos[i]+'o'+str(Position-firstPair)
                rel_pos[i] = rel_pos[i]+'o'+str(Position-secondPair)
            rel_pos[i]=rel_pos[i]+'|'    
        rel_pos[i]=rel_pos[i]+'}'
    i=i+1
data['rel_pos']= rel_pos

In [50]:
#Calculating Interbond Distance
interbond_distance= pd.Series(a.size,dtype= np.str)

for i in range(a.size):
    interbond_distance[i]=''

for i in range(a.size):
    
    if lenB[i]==0:
        interbond_distance[i]='No pair'
        
    if lenB[i]==1:
        interbond_distance[i]='Only One pair'
    
    for r in range(lenB[i]-1):
        split1= re.findall('\d+',data['Disulfide bond'][i][r])
        split2= re.findall('\d+',data['Disulfide bond'][i][r+1])
        interbond_distance[i]= interbond_distance[i]+ str(int(split2[0])-int(split1[1]))+ '|'
        
data['interbond_distance']=distance  

In [51]:
#Intrabond distance formation
intrabond=pd.Series(b.size,dtype=np.str)
for i in range(a.size):
    intrabond[i]=''
j=0;
while j<b.size:
    for k in range(lenB[j]):
        split = re.findall('\d+',b[j][k]) 
        firstPair= int(split[0])
        secondPair=int(split[1])
        intrabond[j]=intrabond[j]+str(secondPair-firstPair)+'|'    
    j=j+1
data['intrabond']=intrabond

In [52]:
#number of pairs of disuphide bonds
data['No. Disulphide bonds']=lenB

In [53]:
#1st disulphide pair to N-terminus distance
first_pos = pd.Series(a.size,dtype=np.int)
for r in range(a.size): 
    split = re.findall('\d+',data['Disulfide bond'][r][0]) 
    firstPair= int(split[0])
    first_pos[r]= firstPair
data['first_position_occurance']=first_pos

In [54]:
#last disulphide pair to C-terminus distance
last_pos= pd.Series(a.size,dtype=np.int)
for r in range(a.size):
    split= re.findall('\d+',data['Disulfide bond'][r][(lenB[r]-1)])
    lastPair= int(split[1])
    last_pos[r] = lastPair 
data['last_position_occurance']=last_pos

In [55]:
data.to_excel('output.xlsx') # Output an Excel File with refined data.

,Entry,Entry name,Protein names,Gene names,Length,Organism,Mass,Status,Disulfide bond,Glycosylation,rel_pos,intrabond,No. Disulphide bonds,first_position_occurance,last_position_occurance,interbond_distance
0,P09564,CD7_HUMAN,T-cell antigen CD7 (GP40) (T-cell leukemia ant...,CD7,240,Homo sapiens (Human),"25,409",reviewed,"[35 142, 48 114]","[45, 96]",45{|i10i-97|o-3o-69|}96{|i61i-46|i48i-18|},107|66|,2,35,114,-94|
1,P09603,CSF1_HUMAN,Macrophage colony-stimulating factor 1 (CSF-1)...,CSF1,554,Homo sapiens (Human),"60,179",reviewed,"[39 122, 63 63, 80 171, 134 178, 189 189, 191 ...","[154, 172]",154{|o115o32|o91o91|i74i-17|i20i-24|o-35o-35|o...,83|0|91|44|0|0|,6,39,191,-59|17|-37|11|2|
2,Q9HCU4,CELR2_HUMAN,Cadherin EGF LAG seven-pass G-type receptor 2 ...,CELSR2 CDHF10 EGFL2 KIAA0279 MEGF3,2923,Homo sapiens (Human),"317,453",reviewed,"[1232 1243, 1237 1274, 1276 1285, 1292 1303, 1...","[486, 557, 701, 1036, 1076, 1182, 1212, 1501, ...",486{|o-746o-757|o-751o-788|o-790o-799|o-806o-8...,11|37|9|11|15|9|11|16|10|26|11|15|9|30|11|15|9...,27,1232,1969,-6|2|7|-6|2|9|-6|2|180|7|-6|2|152|6|-6|2|4|-6|...
3,Q12860,CNTN1_HUMAN,Contactin-1 (Glycoprotein gp135) (Neural cell ...,CNTN1,1018,Homo sapiens (Human),"113,320",reviewed,"[65 114, 158 211, 263 310, 352 391, 436 484, 5...","[208, 258, 338, 457, 473, 494, 521, 591, 933]",208{|o143o94|i50i-3|o-55o-102|o-144o-183|o-228...,49|53|47|39|48|57|,6,65,583,44|52|42|45|42|
4,O15204,ADEC1_HUMAN,ADAM DEC1 (EC 3.4.24.-) (A disintegrin and met...,ADAMDEC1,470,Homo sapiens (Human),"52,775",reviewed,"[328 407, 369 374]","[61, 184, 237, 466]",61{|o-267o-346|o-308o-313|}184{|o-144o-223|o-1...,79|5|,2,328,374,-38|
5,P23946,CMA1_HUMAN,Chymase (EC 3.4.21.39) (Alpha-chymase) (Mast c...,CMA1 CYH CYM,247,Homo sapiens (Human),"27,325",reviewed,"[51 67, 144 209, 175 188]","[80, 103]",80{|o29o13|o-64o-129|o-95o-108|}103{|o52o36|o-...,16|65|13|,3,51,188,77|-34|
6,P01023,A2MG_HUMAN,Alpha-2-macroglobulin (Alpha-2-M) (C3 and PZP-...,A2M CPAMD5 FWP007,1474,Homo sapiens (Human),"163,291",reviewed,"[48 86, 251 299, 269 287, 278 278, 431 431, 47...","[55, 70, 247, 396, 410, 869, 991, 1424]",55{|i7i-31|o-196o-244|o-214o-232|o-223o-223|o-...,38|48|18|0|0|93|176|47|28|36|400|48|115|,13,48,1467,165|-30|-9|153|39|32|-129|132|-2|38|-242|225|
7,P07711,CATL1_HUMAN,Cathepsin L1 (EC 3.4.22.15) (Cathepsin L) (Maj...,CTSL CTSL1,333,Homo sapiens (Human),"37,564",reviewed,"[135 178, 169 211, 269 322]",[221],221{|o86o43|o52o10|o-48o-101|},43|42|53|,3,135,322,-9|58|
8,Q86VB7,C163A_HUMAN,Scavenger receptor cysteine-rich type 1 protei...,CD163 M130,1156,Homo sapiens (Human),"125,451",reviewed,"[76 141, 89 151, 120 130, 184 248, 197 258, 22...","[105, 140, 767, 1027]",105{|i29i-36|i16i-46|o-15o-25|o-79o-143|o-92o-...,65|62|10|64|61|10|64|61|10|64|61|10|64|61|10|6...,26,76,1008,-52|-31|54|-51|-30|53|-51|-30|53|-51|-30|51|-5...
9,O94779,CNTN5_HUMAN,Contactin-5 (Neural recognition molecule NB-2)...,CNTN5,1100,Homo sapiens (Human),"120,686",reviewed,"[123 173, 217 269, 322 369, 411 458, 503 551, ...","[138, 196, 397, 449, 540, 779, 816, 931, 1002]",138{|i15i-35|o-79o-131|o-184o-231|o-273o-320|o...,50|52|47|47|48|57|,6,123,650,44|53|42|45|42|
